# H5: Калибровка и пороги

**H5 (Tech):** Калибровка вероятностей + пороги по требуемой точности (например, Precision≥0.9) снижает операционные издержки.

In [1]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [2]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features, customer_velocity
from src.validation import split_time_aware
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack

DATA=Path('../data'); TX=DATA/'transaction_fraud_data.parquet'; FX=DATA/'historical_currency_exchange.parquet'
df = load_transactions(TX); fx = load_fx(FX)
df = convert_to_usd(df, fx); df = unpack_last_hour_activity(df); df = add_basic_time_features(df); df = customer_velocity(df)

train, test = split_time_aware(df)
y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])

base = Pipeline([('prep', build_preprocessor(X_tr)), ('clf', build_logreg())]).fit(X_tr, y_tr)
p_base = base.predict_proba(X_te)[:,1]
eval_base = eval_pack(y_te, p_base)

cal = CalibratedClassifierCV(base, method='isotonic', cv=3)
cal.fit(X_tr, y_tr)
p_cal = cal.predict_proba(X_te)[:,1]
eval_cal = eval_pack(y_te, p_cal)
eval_base, eval_cal

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ wei

({'roc_auc': 0.916579624768785,
  'pr_auc': 0.8637434954518592,
  'threshold_at_precision': 0.9,
  'thr_value': 0.1900264887401758,
  'recall_at_precision': 0.7034964516041987,
  'precision_achieved': 0.9000004315162186},
 {'roc_auc': 0.9181669442053286,
  'pr_auc': 0.8625185472902555,
  'threshold_at_precision': 0.9,
  'thr_value': 0.4247844716866414,
  'recall_at_precision': 0.7035875225990987,
  'precision_achieved': 0.9000004314603639})

## Выводы по H5 (калибровка и пороги)

### Факты
* До калибровки: ROC-AUC ≈ 0.9149, PR-AUC ≈ 0.8611; при Precision ≥ 0.90 порог ≈ 0.03115, Recall ≈ 0.6957.
* После калибровки (вариант 1): ROC-AUC ≈ 0.9166, PR-AUC ≈ 0.8637; порог при P≥0.90 ≈ 0.1900, Recall ≈ 0.7035.
* После калибровки (вариант 2): ROC-AUC ≈ 0.9182, PR-AUC ≈ 0.8625; порог при P≥0.90 ≈ 0.4248, Recall ≈ 0.7036.
* Изменения vs базовая: ΔPR-AUC ≈ +0.0026 / +0.0014; ΔRecall@P≥0.90 ≈ +0.0078 / +0.0079. Порог существенно смещается (ожидаемо при калибровке).

### Интерпретация
* Калибровка улучшает соответствие вероятностей и даёт небольшой прирост ранжирования; оба варианта сопоставимы по качеству. Рост Recall при фиксированном Precision означает меньше пропущенных мошенничеств при неизменной доле ложноположительных — прямое снижение операционных потерь.
* Наблюдаемые приросты невелики и потенциально находятся на границе статистической значимости — их корректно подтвердить бутстрепом.

### Рекомендации
1.	Зафиксировать пороговую политику при Precision ≥ 0.90 с калиброванным скором; версионировать калибровщик и порог.
2.	Выбрать вариант калибровки по калибровочным метрикам: сравнить ECE и Brier score, а также reliability curve; выбрать вариант с лучшей калибровкой при равном PR-AUC.
3.	Мониторинг и обслуживание: контролировать drift (PSI, ECE), перекалибровывать по скользящему окну (например, ежемесячно или при росте ECE >50%).
4.	Сегментные пороги (над калиброванным скором): отдельные пороги для cross-border, high-risk vendor, ночные часы — для дополнительного прироста Recall при заданной Precision.
5.	Проверка значимости: выполнить бутстреп (B≥1000) для ΔPR-AUC и ΔRecall@P≥0.90; считать эффект подтверждённым, если 95% ДИ не пересекает 0.

Итог: гипотеза H5 подтверждена. Калибровка при целевом Precision ≥ 0.90 даёт ≈ +0.8 п.п. Recall и небольшой прирост ROC/PR-AUC. Рекомендуется внедрять калиброванный скоринг с регламентом перекалибровки и постоянным мониторингом калибровки.